In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage.io import imread
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, accuracy_score

In [ ]:
def get_model():
    logged_model = '../saved_model_for_interpretability'

    model = tf.keras.models.load_model(logged_model)
    return model

In [ ]:
model = get_model()

In [ ]:
data_dir = '/Users/anna.renau/Desktop/hackathon/images_test'

In [ ]:
!find $data_dir  -name ._\* -delete
!find $data_dir  -name .DS\* -delete

---
---
---
# Global analysis

In [ ]:
def build_dataset():
      return tf.keras.preprocessing.image_dataset_from_directory(
          data_dir,
          label_mode="categorical",
          image_size=(384, 384),
          batch_size=1)
    
ds_images = build_dataset()
#classes = ds_images.class_names
classes = ['bathroom', 'bedroom', 'dinning', 'frontal', 'kitchen', 'livingroom']

# Hemos incluido la normalización (Resize y Rescaling) como capas dentro del modelo.
# normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1. / 255)
# ds_images = ds_images.map(lambda images, labels: (normalization_layer(images), labels))

In [ ]:
y_pred = []
y_true = []
for x, y in ds_images:
    image = x[0, :, :, :]
    pred = classes[np.argmax(model.predict(np.expand_dims(image, axis=0)))]
    true_value = classes[np.argmax(y[0])]
    y_pred.append(pred)
    y_true.append(true_value)

    if pred != true_value:
        plt.imshow(image.numpy().astype("uint8"))
        plt.title(f'pred: {pred} - true: {true_value} ')
        plt.show()

In [ ]:
y_true_freqs = np.unique(y_true, return_counts=True)
y_pred_freqs = np.unique(y_pred, return_counts=True)

fig, ax = plt.subplots(figsize = (7, 5))
ax.bar(y_true_freqs[0], y_true_freqs[1], alpha = 0.3, label = 'y_true')
ax.bar(y_pred_freqs[0], y_pred_freqs[1], alpha = 0.3, label = 'y_pred')
ax.legend()

In [ ]:
print('Accuracy:', accuracy_score(y_true, y_pred))
print('F1 score:', f1_score(y_true, y_pred, average=None))
print('Precision:', precision_score(y_true, y_pred, average=None))
print('Recall:', recall_score(y_true, y_pred, average=None))

fig, ax = plt.subplots(figsize=(10,10))
cm = confusion_matrix(y_true, y_pred, labels=classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=classes)
disp.plot(ax=ax)
plt.show()